# **PROBLEM STATEMENT AND DATA DESCRIPTION:**

**Hostile Post Detection in Hindi** :
       
This subtask focuses on a variety of **hostile posts in Hindi** Devanagari script **collected from Twitter and Facebook**. The set of valid categories are fake news, hate speech, offensive, defamation, and non-hostile posts. It is a **multi-label multi-class classification problem** where each post can belong to one or more of these hostile classes. However, the non-hostile posts cannot be grouped with any other class. The evaluation of this subtask will be two-dimensional as follows:

**Coarse-grained evaluation:** It is a binary evaluation of hostile vs non-hostile posts.

**Fine-grained evaluation:** It is a fine-grained evaluation of the hostile classes.

**Definitions of the class labels:**

**Fake News:** A claim or information that is verified to be not true.

**Hate Speech:** A post targeting a specific group of people based on their ethnicity, religious beliefs, geographical belonging, race, etc., with malicious intentions of spreading hate or encouraging violence.

**Offensive:** A post containing profanity, impolite, rude, or vulgar language to insult a targeted individual or group.

**Defamation:** A mis-information regarding an individual or group.

**Non-hostile:** A post without any hostility.

**Evaluation Metric:** 

The official evaluation metric for the shared task is  **weighted-average F1 score**.



In [ ]:
# to import datasets from google drive 
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **IMPORTING THE LIBRARIES :**

In [ ]:
pip install polyglot

     |████████████████████████████████| 133kB 8.8MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52558 sha256=9cfd5918d7595104981985ab209e3d082db566a8cee3362843df03a47e25c644
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot


In [ ]:
pip install icu

     |████████████████████████████████| 51kB 4.2MB/s 


In [ ]:
pip install -U git+https://github.com/aboSamoor/polyglot.git@master

  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /tmp/pip-req-build-zl84o55s
  Running command git clone -q https://github.com/aboSamoor/polyglot.git /tmp/pip-req-build-zl84o55s
     |████████████████████████████████| 235kB 7.6MB/s 
     |████████████████████████████████| 41.4MB 111kB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=70644 sha256=4aa977c4c51f5e7b08d3d51f8bd24e651523773006cfc3326a64e0852a10bea6
  Stored in directory: /tmp/pip-ephem-wheel-cache-qcue7ruf/wheels/42/d9/73/345c7ae8554299ff8b31635d64eb8455fd591385fa734cdbef
  Created wheel for PyICU: filename=PyICU-2.6-cp36-cp36m-linux_x86_64.whl size=1288406 sha256=83bc59ce0a58051bc25bc58830179bb1761291ddb648eba772d33a5d951abc8c
  Stored in directory: /root/.cache/pip/wheels/31/21/2f/1c91831e8a93537ab21f6b4b935781b681104635fdb0315791
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833486 sha256=d63250037d3f93f8e84025a8a24fef4cab44

In [ ]:
pip install snowballstemmer

In [ ]:
# importing the libraries 
import os
import itertools
import numpy as np 
import pandas as pd 
import re 
import math as m 
import matplotlib.pyplot as plt
import seaborn as sns

import polyglot
from polyglot.text import Text,Word
from polyglot.downloader import downloader
import snowballstemmer
from snowballstemmer import HindiStemmer

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report as Report

import warnings 
warnings.filterwarnings("ignore")

In [ ]:
# to display entire rows and columns of dataframe 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# **LOADING THE DATASET**  

In [ ]:
# read the train data 
train_df=pd.read_excel('/content/gdrive/My Drive/constraint_Hindi_Train.xlsx')
train_df.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"hate,offensive"
1,2,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs",non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर #LIVE : https://t.co/G945HvzM0Z https://t.co/KfH7xF1IdM",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे. https://t.co/4e6lysg0VR,non-hostile


In [ ]:
# read the validate data 
validate_df=pd.read_excel('/content/gdrive/My Drive/Constraint_Hindi_Valid.xlsx')
validate_df.head()

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,defamation
2,3,"कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया\nhttps://t.co/9rlQowAsFh #Delhi @ArvindKejriwal @rajnathsingh @AmitShah @DRDO_India @WHO @crpfindia @ITBP_official",non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए\nhttps://t.co/ZvKgxk6dbd,non-hostile
4,5,"यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव ने किया लाखों का गबन, मुर्दों के नाम पर बनवा डाले शौचालय\n\n#UP\nhttps://t.co/hxM1uNNmX2",non-hostile


In [ ]:
# data check
def about_the_data(dataset):
  """
  input  : dataset 
  prints : first 5 rows of dataset , shape of dataset , information about output class label of dataset
  """
  print(dataset.head())
  print("--------------------------------------------------------------------------------------------")
  print("Shape of dataset : \n")
  print(dataset.shape)
  print("--------------------------------------------------------------------------------------------")
  print(dataset['Labels Set'].unique())
  print("--------------------------------------------------------------------------------------------")
  print(dataset['Labels Set'].value_counts())

In [ ]:
# details about train data 
print("TRAIN DATASET : ")
print("----------------------------------------------------------------------------------------------")
about_the_data(train_df)

TRAIN DATASET : 
----------------------------------------------------------------------------------------------
   Unique ID  \
0  1           
1  2           
2  3           
3  4           
4  5           

                                                                                                                                                                                                                                                                                                                    Post  \
0  मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏                                                                                                                                                                                                                  
1  सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs                 

In [ ]:
# details about validate data 
print("VALIDATION DATASET : ")
print("----------------------------------------------------------------------------------------------")
about_the_data(validate_df)

VALIDATION DATASET : 
----------------------------------------------------------------------------------------------
   Unique ID  \
0  1           
1  2           
2  3           
3  4           
4  5           

                                                                                                                                                                                                                                                                                        Post  \
0  दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।\n\nउनका निधन हम सब के लिए अपूरणीय क्षति है।\nईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं।\nऊं शांति!!!                                                                       
1  भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी  करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा 

# **DATA PRE-PROCESSING:**

### **Remove Punctuations and Special Characters:**

In [ ]:
# this function removes punctuations and special characters in the dataset
def PunctuationsAndSpecialSymbolsRemover(data):
  """
  input   : post data from train or validation or test data (list)
  returns : cleaned post with punctuations and special symbols removed 
  """
  non_needed_symbols = '''!()-…[]{};₹:“।|”'"\,<>./?@#$%^&*+_~'''
  cleaned_list=[]
  for line in data:
    cleaned_text=""
    for x in line:
      if x in non_needed_symbols:
        cleaned_text=cleaned_text+" "
      elif x not in non_needed_symbols:
        cleaned_text=cleaned_text+x
    cleaned_list.append(cleaned_text)
  return cleaned_list

### **Remove New Lines:** 

In [ ]:
# remove new lines notation "\n"
def remove_newlines(data):
  """
  input   : post data from train or validation or test data (list)
  returns : cleaned data with newlines ('\n') symbols removed 
  """
  modified_data=[]
  for line in data:
    new_line=re.compile("["u"\u000A" #newlines
                           "]+", flags=re.UNICODE)
    cleaned_text=new_line.sub(r'', line)
    modified_data.append(cleaned_text)
  return modified_data

### **Remove HyperLinks:** 

In [ ]:
# this function removes entire hyperlink 
def HyperLinkRemover(data):
  """
  input   : post data from train or validation or test data (list)
  returns : cleaned data with hyperlinks in the posts removed
  """
  non_hyperlink_text=[]
  for line in data:
    cleaned_text=re.sub(r"http\S+", "", line)
    non_hyperlink_text.append(cleaned_text)
  return non_hyperlink_text

### **Remove the Emojis:**

In [ ]:
# remove the emojis from the post 
def emojiRemover(data):
  """
  input   : post data from train or validation or test data (list)
  returns : cleaned data with all the emojis used in the post gets removed
  """
  emoji_less_data=[]
  for line in data:
    emoji_pattern=re.compile("["
                           u"\U0001F601-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6C0"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U000024C2-\U0001F251"  # Enclosed characters
                           u"\U0001F600-\U0001F636"  # Additional emoticons
                           u"\U0001F681-\U0001F6C5"  # Additional transport & map symbols
                           u"\U0001F30D-\U0001F567"  # Additional
                            u"\U0001f926-\U0001f937"
                            u"\u000A" #newlines
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               u"\u2022" # bullets 
                           "]+", flags=re.UNICODE)
    cleaned_text=emoji_pattern.sub(r'', line)
    #cleaned_text=uncategorized_emojis(cleaned_text)
    emoji_less_data.append(cleaned_text)
  return emoji_less_data

### **Remove Stopwords :**

[STOP WORDS IN HINDI](https://data.mendeley.com/datasets/bsr3frvvjc/1)

In [ ]:
hindi_stop_words = []
with open('final_stopwords.txt','r') as file: 
  for word in file: 
    # reading each word         
    for w in word.split(): 
      hindi_stop_words.append(w)
print("Number of Hindi Stopwords : ",len(hindi_stop_words))

Number of Hindi Stopwords :  291


In [ ]:
def remove_stopwords(data):
  """
  input   : post data from train or validation or test data (list)
  returns : cleaned data with all the stopwords removed 
  """
  stop_words_removed_data=[]
  for line in data:
    cleaned_data=""
    for word in line.split(" "):
      if word not in hindi_stop_words:
        cleaned_data=cleaned_data+word+" "
    stop_words_removed_data.append(cleaned_data)
  return stop_words_removed_data

### **Create Datasets:**

##### **Cleaned Datasets:**

In [ ]:
# function to do data-preprocessing 
def pre_processesing(data):
  """
  input   : post data from train or validation or test data (list)
  returns : cleaned data with all pre-processesing steps performed 
  """
  data=HyperLinkRemover(data)
  data=PunctuationsAndSpecialSymbolsRemover(data)
  data=emojiRemover(data)
  data=remove_newlines(data)
  #data=remove_stopwords(data)
  return data

# function to return the cleaned data 
def cleaned_xlsx(dataset,function=pre_processesing):
  """
  input   : dataset 
  function: default - pre_processesing function
                can be changed to either emoji remover,new liner remover ,hyperlink remover etc.
  returns : dataset with post content cleaned 
  """
  cleaned_df_hindi=pd.DataFrame()
  cleaned_df_hindi['Unique ID']=dataset['Unique ID']
  data=dataset['Post']
  cleaned_df_hindi['Post']=function(data)
  cleaned_df_hindi['Labels Set']=dataset['Labels Set']
  return cleaned_df_hindi

In [ ]:
"""
train_emoji=cleaned_xlsx(train_df,emojiRemover)
train_punctuations=cleaned_xlsx(train_df,PunctuationsAndSpecialSymbolsRemover)
train_hyperlinks=cleaned_xlsx(train_df,HyperLinkRemover)
train_newlines=cleaned_xlsx(train_df,remove_newlines)
validate_emoji=cleaned_xlsx(validate_df,emojiRemover)
validate_punctuations=cleaned_xlsx(validate_df,PunctuationsAndSpecialSymbolsRemover)
validate_hyperlinks=cleaned_xlsx(validate_df,HyperLinkRemover)
validate_newlines=cleaned_xlsx(validate_df,remove_newlines)
"""

'\ntrain_emoji=cleaned_xlsx(train_df,emojiRemover)\ntrain_punctuations=cleaned_xlsx(train_df,PunctuationsAndSpecialSymbolsRemover)\ntrain_hyperlinks=cleaned_xlsx(train_df,HyperLinkRemover)\ntrain_newlines=cleaned_xlsx(train_df,remove_newlines)\nvalidate_emoji=cleaned_xlsx(validate_df,emojiRemover)\nvalidate_punctuations=cleaned_xlsx(validate_df,PunctuationsAndSpecialSymbolsRemover)\nvalidate_hyperlinks=cleaned_xlsx(validate_df,HyperLinkRemover)\nvalidate_newlines=cleaned_xlsx(validate_df,remove_newlines)\n'

In [ ]:
# cleaned train dataset 
cleaned_train_df=cleaned_xlsx(train_df)

# cleaned validate dataset
cleaned_validate_df=cleaned_xlsx(validate_df)

In [ ]:
x=cleaned_train_df.head()
x

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई नई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े,non-hostile
2,3,सुशांत ने जो बिजनेस डील 9 जून को की थी वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई देखिए पूछता है भारत अर्नब के साथ रिपब्लिक भारत पर LIVE,non-hostile
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,"defamation,offensive"
4,5,unlock4guidelines अनलॉक 4 के लिए गाइडलाइन्स जारी 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत कंटेनमेंट जोन में कोई छूट नहीं सिनेमाहॉल अभी बंद रहेंगे 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे,non-hostile


In [ ]:
cleaned_train_df.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,"hate,offensive"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई नई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े,non-hostile
2,3,सुशांत ने जो बिजनेस डील 9 जून को की थी वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई देखिए पूछता है भारत अर्नब के साथ रिपब्लिक भारत पर LIVE,non-hostile
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,"defamation,offensive"
4,5,unlock4guidelines अनलॉक 4 के लिए गाइडलाइन्स जारी 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत कंटेनमेंट जोन में कोई छूट नहीं सिनेमाहॉल अभी बंद रहेंगे 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे,non-hostile


In [ ]:
cleaned_validate_df.head()

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा उनका निधन हम सब के लिए अपूरणीय क्षति है ईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें शोक संतप्त परिजनों के प्रति संवेदनाएं ऊं शांति,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,defamation
2,3,कोरोना से निपटने की तैयारी दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू राजनाथ शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया Delhi ArvindKejriwal rajnathsingh AmitShah DRDO India WHO crpfindia ITBP official,non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले शिक्षा नीति में सरकार का दखल कम होना चाहिए,non-hostile
4,5,यूपी गाजीपुर में Toilet घोटाला प्रधान व सचिव ने किया लाखों का गबन मुर्दों के नाम पर बनवा डाले शौचालय UP,non-hostile


##### **Creating the Dataset for Binary Classification:** 

In [ ]:
# binary classification labels either hostile or non - hostile
def binary_labels(data):
  """
  input   :output label set (list) 
  returns :list of output labels which has non-hostile if given input is non-hostile else hostile 
  """
  labels=['non-hostile']
  modified_labels=[]
  for x in data:
    if x not in labels:
      modified_labels.append("hostile")
    else:
      modified_labels.append("non-hostile")
  return modified_labels

# binary classification dataset is created using this function 
def make_binary_class_dataset(dataset):
  """
  input   : dataset 
  returns : dataset with output labels changed to either hostile or non-hostile for binary classification 
  """
  binary_df_hindi=pd.DataFrame()
  binary_df_hindi['Unique ID']=dataset['Unique ID']
  binary_df_hindi['Post']=dataset['Post']
  data=dataset['Labels Set']
  binary_df_hindi['Labels Set']=binary_labels(data)
  return binary_df_hindi

In [ ]:
# binary classification train data 
binary_train_df=make_binary_class_dataset(cleaned_train_df)
binary_train_df.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,hostile
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई नई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े,non-hostile
2,3,सुशांत ने जो बिजनेस डील 9 जून को की थी वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई देखिए पूछता है भारत अर्नब के साथ रिपब्लिक भारत पर LIVE,non-hostile
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,hostile
4,5,unlock4guidelines अनलॉक 4 के लिए गाइडलाइन्स जारी 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत कंटेनमेंट जोन में कोई छूट नहीं सिनेमाहॉल अभी बंद रहेंगे 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे,non-hostile


In [ ]:
# binary classification validate data 
binary_validate_df=make_binary_class_dataset(cleaned_validate_df)
binary_validate_df.head()

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा उनका निधन हम सब के लिए अपूरणीय क्षति है ईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें शोक संतप्त परिजनों के प्रति संवेदनाएं ऊं शांति,non-hostile
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,hostile
2,3,कोरोना से निपटने की तैयारी दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू राजनाथ शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया Delhi ArvindKejriwal rajnathsingh AmitShah DRDO India WHO crpfindia ITBP official,non-hostile
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले शिक्षा नीति में सरकार का दखल कम होना चाहिए,non-hostile
4,5,यूपी गाजीपुर में Toilet घोटाला प्रधान व सचिव ने किया लाखों का गबन मुर्दों के नाम पर बनवा डाले शौचालय UP,non-hostile


In [ ]:
# details about the binary_train data 
about_the_data(binary_train_df)

   Unique ID  \
0  1           
1  2           
2  3           
3  4           
4  5           

                                                                                                                                                                                                                                                                                 Post  \
0  मेरे देश के हिन्दु बहुत निराले है  कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है  जय श्री राम                                                                                                                                                                                      
1  सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई नई स्कीमें लाती रहती है  ताकि उन पर ज्यादा आर्थिक बोझ न पड़े                                                                                                                                                                      
2  सुशांत ने जो बिजनेस डील 9 जून को की थी  वो डील दीपेश को स

In [ ]:
# details about the binary_validate data 
about_the_data(binary_validate_df)

   Unique ID  \
0  1           
1  2           
2  3           
3  4           
4  5           

                                                                                                                                                                                                                                                                                Post  \
0  दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा उनका निधन हम सब के लिए अपूरणीय क्षति है ईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें  शोक संतप्त परिजनों के प्रति संवेदनाएं ऊं शांति                                                                          
1  भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी  करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो   
2  कोरोना से निपटने की तैयारी   दिल्ली में 10 हजार बेड वाला दुन

##### **Dataset with output label as one-hot vectors:**





In [ ]:
# convert the output labels to one hot vectors 
def vector_labels(data,h=1):
  """
  input   : data(list) , h=1
  returns : one-hot encoded vectors for entire dataset by default and if h=0 it returns it returns one-hot encoded vectors
            for hostile data only 
  """
  final_labels_vectors=[]
  for x in data:
    vec=[0]*5
    l=list(x.split(','))
    for y in l:
      if vec[0]!=1 and y=='non-hostile':
        vec[0]=1
      if vec[1]!=1 and y=="defamation":
        vec[1]=1
      if vec[2]!=1 and y=="fake":
        vec[2]=1
      if vec[3]!=1 and y=="hate":
        vec[3]=1
      if vec[4]!=1 and y=="offensive":
        vec[4]=1
    if h==1:
      final_labels_vectors.append(vec)
    else:
      final_labels_vectors.append(vec[1:])
  return final_labels_vectors

# function to return the dataset with output as one-hot vectors 
def labelsToVectors(dataset,h=1):
  """
  input   : dataset 
      h   : default 1 --- 5 labels converting to 5 hot vectors
            if 0 --- 4 labels converting to 4 hot vectors
  returns : dataset with output labels modified to one-hot vectors (h=1 for entire data and h=0 for hostile data only)
  """
  labelsvec_df_hindi=pd.DataFrame()
  labelsvec_df_hindi['Unique ID']=dataset['Unique ID']
  labelsvec_df_hindi['Post']=dataset['Post']
  data=dataset['Labels Set']
  if h==1:
    labelsvec_df_hindi['Labels Set']=vector_labels(data)
  else:
    labelsvec_df_hindi['Labels Set']=vector_labels(data,0)
  return labelsvec_df_hindi

# 5 one-hot vectors 
outputlabel5_train_df=labelsToVectors(cleaned_train_df)
outputlabel5_validate_df=labelsToVectors(cleaned_validate_df)

# 4 one-hot vectors 
outputlabel4_train_df=labelsToVectors(cleaned_train_df[cleaned_train_df["Labels Set"] != 'non-hostile'].reset_index(drop=True),0)
outputlabel4_validate_df=labelsToVectors(cleaned_validate_df[cleaned_validate_df["Labels Set"] != 'non-hostile'].reset_index(drop=True),0)

In [ ]:
# train data with 5 hot vectors 
outputlabel5_train_df.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,"[0, 0, 0, 1, 1]"
1,2,सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई नई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े,"[1, 0, 0, 0, 0]"
2,3,सुशांत ने जो बिजनेस डील 9 जून को की थी वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई देखिए पूछता है भारत अर्नब के साथ रिपब्लिक भारत पर LIVE,"[1, 0, 0, 0, 0]"
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,"[0, 1, 0, 0, 1]"
4,5,unlock4guidelines अनलॉक 4 के लिए गाइडलाइन्स जारी 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत कंटेनमेंट जोन में कोई छूट नहीं सिनेमाहॉल अभी बंद रहेंगे 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे,"[1, 0, 0, 0, 0]"


In [ ]:
# validate data with 5 hot vectors 
outputlabel5_validate_df.head()

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा उनका निधन हम सब के लिए अपूरणीय क्षति है ईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें शोक संतप्त परिजनों के प्रति संवेदनाएं ऊं शांति,"[1, 0, 0, 0, 0]"
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,"[0, 1, 0, 0, 0]"
2,3,कोरोना से निपटने की तैयारी दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू राजनाथ शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया Delhi ArvindKejriwal rajnathsingh AmitShah DRDO India WHO crpfindia ITBP official,"[1, 0, 0, 0, 0]"
3,4,गवर्नर कॉन्फ्रेंस में PM मोदी बोले शिक्षा नीति में सरकार का दखल कम होना चाहिए,"[1, 0, 0, 0, 0]"
4,5,यूपी गाजीपुर में Toilet घोटाला प्रधान व सचिव ने किया लाखों का गबन मुर्दों के नाम पर बनवा डाले शौचालय UP,"[1, 0, 0, 0, 0]"


In [ ]:
# train data with 4 hot vectors 
outputlabel4_train_df.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,"[0, 0, 1, 1]"
1,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,"[1, 0, 0, 1]"
2,6,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अजर‌ मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करें विक्रम शर्मा,"[0, 1, 0, 0]"
3,11,RT Pb swain मोदीजी और जब सारा देश सेना के साथ खडी है पर दो सयाने विदेश मे पडे है इसलिए बोलते हैं विदेशी मां का बेटा कभी देशभक्त न,"[0, 0, 1, 0]"
4,12,RT ShilpiSinghINC 48000 घरों को तोड़ने का आदेश आया है किसी को कोई फर्क़ नहीं पड़ता अमीरों के 5 में से 1 मकान के किसी अवैध हिस्से भी एक,"[0, 0, 0, 1]"


In [ ]:
# validate data with 4 hot vectors 
outputlabel4_validate_df.head()

,Unique ID,Post,Labels Set
0,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,"[1, 0, 0, 0]"
1,8,अद्भुत जो वामपंथी कहते है कि महाभारत का युद्ध काल्पनिक है और सनातन धर्म कुछ है ही नही उनके मुंह पर विदेशी पुरातत्व विभाग ने जोरदार तमाचा मार दिया है कुरूक्षेत्र के पास खुदाई करते समय विदेशी पुरातत्व विशेषज्ञों को एक 80 फुट की लम्बाई के मानव कंकाल के अवषेश मिले जो महाभारत के भीम के पुत्र घटोत्कच के वर्णन के समान है और हम भारत वासियों को महाभारत ही कहानी काल्पनीक लगती है इसे डिस्कवरी चैनल ने प्रसारित किया है जय हो सनातन धर्म की जय हो जय श्री राम जय श्री कृष्ण,"[0, 1, 0, 0]"
2,13,भाई जाके हिन्दू भाइयों की मदद कर जिनकी नौकरी चली गई जो आत्महत्या कर रहे है अब ये मत पूछना तूने कितनी की मदद की,"[0, 0, 0, 1]"
3,14,यह दुर्घटना कन्नूर अंतर्राष्ट्रीय हवाई अड्डे में हुई जब एक व्यक्ति पावर बैंक का उपयोग करके अपने सेल फोन को चार्ज कर रहा था और आग लग गई जागरूकता बढ़ाने के लिए वीडियो को शेयर करें,"[0, 1, 0, 0]"
4,16,रेट फिक्स हैकृप्या मोल भाव न करें शाहीन बाग़ प्रोटेस्ट,"[0, 0, 1, 1]"


##### **Datasets with Fake,Non_Fake & Offensive,Non_Offensive & Hate,Non_Hate & Defamation,Non_Defamation :**

In [ ]:
# hostile train data 
hostile_train=cleaned_train_df[cleaned_train_df['Labels Set'] !="non-hostile"]
hostile_train.head() 

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,"hate,offensive"
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,"defamation,offensive"
5,6,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अजर‌ मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करें विक्रम शर्मा,fake
10,11,RT Pb swain मोदीजी और जब सारा देश सेना के साथ खडी है पर दो सयाने विदेश मे पडे है इसलिए बोलते हैं विदेशी मां का बेटा कभी देशभक्त न,hate
11,12,RT ShilpiSinghINC 48000 घरों को तोड़ने का आदेश आया है किसी को कोई फर्क़ नहीं पड़ता अमीरों के 5 में से 1 मकान के किसी अवैध हिस्से भी एक,offensive


In [ ]:
# hostile validate data 
hostile_validate=cleaned_validate_df[cleaned_validate_df['Labels Set'] !="non-hostile"]
hostile_validate.head() 

,Unique ID,Post,Labels Set
1,2,भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,defamation
7,8,अद्भुत जो वामपंथी कहते है कि महाभारत का युद्ध काल्पनिक है और सनातन धर्म कुछ है ही नही उनके मुंह पर विदेशी पुरातत्व विभाग ने जोरदार तमाचा मार दिया है कुरूक्षेत्र के पास खुदाई करते समय विदेशी पुरातत्व विशेषज्ञों को एक 80 फुट की लम्बाई के मानव कंकाल के अवषेश मिले जो महाभारत के भीम के पुत्र घटोत्कच के वर्णन के समान है और हम भारत वासियों को महाभारत ही कहानी काल्पनीक लगती है इसे डिस्कवरी चैनल ने प्रसारित किया है जय हो सनातन धर्म की जय हो जय श्री राम जय श्री कृष्ण,fake
12,13,भाई जाके हिन्दू भाइयों की मदद कर जिनकी नौकरी चली गई जो आत्महत्या कर रहे है अब ये मत पूछना तूने कितनी की मदद की,offensive
13,14,यह दुर्घटना कन्नूर अंतर्राष्ट्रीय हवाई अड्डे में हुई जब एक व्यक्ति पावर बैंक का उपयोग करके अपने सेल फोन को चार्ज कर रहा था और आग लग गई जागरूकता बढ़ाने के लिए वीडियो को शेयर करें,fake
15,16,रेट फिक्स हैकृप्या मोल भाव न करें शाहीन बाग़ प्रोटेस्ट,"hate,offensive"


In [ ]:
# creating dataset with each label passed through the function 
def vector_labels(data,col,neg):
  """
  input  : data: output label data (list)
           col : true label 
           neg : false label 
  return : list of output labels (if col in data return col else return neg for each entry in list od ata)
  """
  final_labels_vectors=[]
  for x in data:
    l=list(x.split(','))
    f=0
    for y in l:
      if y==col:
        final_labels_vectors.append(col)
        f=1
      if f==1:
        continue
    if f==0:
      final_labels_vectors.append(neg)
  return final_labels_vectors

# creating dataset by passing the given hostile labels
def hostile_label(dataset,col,neg):
  """
  input   : dataset : hostile data (entries which doesn't contain non-hostile)
                col : true label 
                neg : false label
  returns : modified dataset with output label should be either col or neg 
  """
  labelsvec_df_hindi=pd.DataFrame()
  labelsvec_df_hindi['Unique ID']=dataset['Unique ID']
  labelsvec_df_hindi['Post']=dataset['Post']
  data=dataset['Labels Set']
  labelsvec_df_hindi['Labels Set']=vector_labels(data,col,neg)
  return labelsvec_df_hindi

# fake vs non_fake
fake_train=hostile_label(hostile_train,"fake","non_fake")
fake_validate=hostile_label(hostile_validate,"fake","non_fake")

# hate vs non_hate 
hate_train=hostile_label(hostile_train,"hate","non_hate")
hate_validate=hostile_label(hostile_validate,"hate","non_hate")

# defamation vs non_defamation 
defamation_train=hostile_label(hostile_train,"defamation","non_defamation")
defamation_validate=hostile_label(hostile_validate,"defamation","non_defamation")

# offensive vs non_offensive 
offensive_train=hostile_label(hostile_train,"offensive","non_offensive")
offensive_validate=hostile_label(hostile_validate,"offensive","non_offensive")

In [ ]:
# fake_train data 
fake_train.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,non_fake
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,non_fake
5,6,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अजर‌ मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करें विक्रम शर्मा,fake
10,11,RT Pb swain मोदीजी और जब सारा देश सेना के साथ खडी है पर दो सयाने विदेश मे पडे है इसलिए बोलते हैं विदेशी मां का बेटा कभी देशभक्त न,non_fake
11,12,RT ShilpiSinghINC 48000 घरों को तोड़ने का आदेश आया है किसी को कोई फर्क़ नहीं पड़ता अमीरों के 5 में से 1 मकान के किसी अवैध हिस्से भी एक,non_fake


In [ ]:
# offensive train data
offensive_train.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,offensive
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,offensive
5,6,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अजर‌ मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करें विक्रम शर्मा,non_offensive
10,11,RT Pb swain मोदीजी और जब सारा देश सेना के साथ खडी है पर दो सयाने विदेश मे पडे है इसलिए बोलते हैं विदेशी मां का बेटा कभी देशभक्त न,non_offensive
11,12,RT ShilpiSinghINC 48000 घरों को तोड़ने का आदेश आया है किसी को कोई फर्क़ नहीं पड़ता अमीरों के 5 में से 1 मकान के किसी अवैध हिस्से भी एक,offensive


In [ ]:
# hate train data 
hate_train.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,hate
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,non_hate
5,6,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अजर‌ मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करें विक्रम शर्मा,non_hate
10,11,RT Pb swain मोदीजी और जब सारा देश सेना के साथ खडी है पर दो सयाने विदेश मे पडे है इसलिए बोलते हैं विदेशी मां का बेटा कभी देशभक्त न,hate
11,12,RT ShilpiSinghINC 48000 घरों को तोड़ने का आदेश आया है किसी को कोई फर्क़ नहीं पड़ता अमीरों के 5 में से 1 मकान के किसी अवैध हिस्से भी एक,non_hate


In [ ]:
# defamation train data
defamation_train.head()

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,non_defamation
3,4,prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,defamation
5,6,चीन ने UN में तर्क दिया की भारत का विपक्ष ही अजर‌ मसुद को आतंकी नहीं मानता तो हम कैसे माने चुल्लू भर मूत्र में डूब मरो गद्दारों अब यह भारत के लोगों को सोचना है कि वो विपक्ष को वोट क्यों करें विक्रम शर्मा,non_defamation
10,11,RT Pb swain मोदीजी और जब सारा देश सेना के साथ खडी है पर दो सयाने विदेश मे पडे है इसलिए बोलते हैं विदेशी मां का बेटा कभी देशभक्त न,non_defamation
11,12,RT ShilpiSinghINC 48000 घरों को तोड़ने का आदेश आया है किसी को कोई फर्क़ नहीं पड़ता अमीरों के 5 में से 1 मकान के किसी अवैध हिस्से भी एक,non_defamation


### **Named Entity Recognition :**

In [ ]:
# embedding package for english 
downloader.download("embeddings2.en")

[polyglot_data] Downloading package embeddings2.en to
[polyglot_data]     /root/polyglot_data...


True

In [ ]:
# named entity recognition package for english 
downloader.download("ner2.en")

[polyglot_data] Downloading package ner2.en to /root/polyglot_data...


True

In [ ]:
# embedding package for hindi
downloader.download("embeddings2.hi")

[polyglot_data] Downloading package embeddings2.hi to
[polyglot_data]     /root/polyglot_data...


True

In [ ]:
# named entity recognition package for english 
downloader.download("ner2.hi")

[polyglot_data] Downloading package ner2.hi to /root/polyglot_data...


True

In [ ]:
# this function removes named entity for the given language code 
def named_entity_recognition(data,code):
  """
  input   : data : post (not lists of posts only one post)
            code : language code either english("en") or hindi("hi") 
  returns : post data with NER removed for the language code chosen
  """
  l=[]
  entity=Text(data,hint_language_code=code).entities
  ner_list=[]
  for o in entity:
    ner_list.append(" ".join(o))  
  ner_list2=[]
  for item in ner_list:
    ner_list2.extend(item.split()) 
  new=list(data.split(" "))
  for y in new:
    if y in ner_list2:
      new.remove(y)
  l.append(new)
  return " ".join(l[0])

# creates the new dataset after removing the named entities 
def dataset_after_NER(dataset,code):
  """
  input   : dataset : cleaned binary label data 
               code : language code either english("en") or hindi("hi")
  returns : cleaned data after named entity recognitions removed for the language code specified 
  """
  cleaned_df_hindi=pd.DataFrame()
  cleaned_df_hindi['Unique ID']=dataset['Unique ID']
  data=dataset['Post']
  out=[]
  for d in data:
    out.append(named_entity_recognition(d,code))
  cleaned_df_hindi['Post']=out
  cleaned_df_hindi['Labels Set']=dataset['Labels Set']
  return cleaned_df_hindi

# ner on train data
ner_binary_train=dataset_after_NER(binary_train_df,'hi') # hindi named entity removed 
ner_binary_train=dataset_after_NER(ner_binary_train,'en') # english named entity removed 

# ner on validate data 
ner_binary_validate=dataset_after_NER(binary_validate_df,'hi')
ner_binary_validate=dataset_after_NER(ner_binary_validate,'en')

In [ ]:
ner_binary_train.head()

,Unique ID,Post,Labels Set
0,1,देश के हिन्दु बहुत निराले है कुछ तो पक्के भक्त है और कुछ के साले है जय राम,hostile
1,2,हमेशा से किसानों की कमाई को बढ़ाने के लिए नई नई स्कीमें लाती रहती है ताकि उन पर ज्यादा आर्थिक बोझ न पड़े,non-hostile
2,3,जो बिजनेस डील 9 जून को की थी वो डील दीपेश को की हत्या के दिन ही क्यों याद आई देखिए पूछता है अर्नब के साथ रिपब्लिक पर LIVE,non-hostile
3,4,छाप कमिने लोग को यह कहते है की संविधान सबको बराबर अधिकार देता है सच्चाई यह है कि यह बराबर अधिकार नहीं देता है,hostile
4,5,अनलॉक 4 के लिए गाइडलाइन्स जारी 7 सितंबर से में मेट्रो सेवा शुरु होगी 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत कंटेनमेंट जोन में कोई छूट नहीं सिनेमाहॉल अभी बंद रहेंगे 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे,non-hostile


In [ ]:
ner_binary_validate.head()

,Unique ID,Post,Labels Set
0,1,इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा उनका निधन हम सब के लिए अपूरणीय क्षति है ईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें शोक संतप्त परिजनों के प्रति संवेदनाएं ऊं शांति,non-hostile
1,2,जनता rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss वाले की जासूस में कौन है प्रदेश में कौन है राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो,hostile
2,3,से निपटने की तैयारी में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू शाह ने के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया ArvindKejriwal rajnathsingh AmitShah crpfindia ITBP official,non-hostile
3,4,कॉन्फ्रेंस में PM बोले शिक्षा नीति में सरकार का दखल कम होना चाहिए,non-hostile
4,5,में Toilet घोटाला प्रधान व सचिव ने किया लाखों का गबन मुर्दों के नाम पर बनवा डाले शौचालय UP,non-hostile


### **Stemming:**

In [ ]:
obj=HindiStemmer() # stemmer used for hindi language 

# stemmed words in hindi is removed and it is done using hindi stemmer 
def stem_word_removal(data):
  """
  input   : data : post data which will contain a single post with so many words 
  returns : data with each word after being stemmed
  """
  l=[]
  for word in data.split(" "): # take each word from post and if greater than 1 stem it 
    if len(word)>1:
      stem_word=obj.stemWord(word)
      l.append(stem_word)
  out=" ".join(l)
  return out

# creating the dataset after stemming 
def dataset_after_stemming(dataset):
  """
  input   : dataset : cleaned binary label data 
  returns : cleaned data after words in posts are stemmed
  """
  cleaned_df_hindi=pd.DataFrame()
  cleaned_df_hindi['Unique ID']=dataset['Unique ID']
  data=dataset['Post']
  out=[]
  for d in data:
    out.append(stem_word_removal(d))
  cleaned_df_hindi['Post']=out
  cleaned_df_hindi['Labels Set']=dataset['Labels Set']
  return cleaned_df_hindi

In [ ]:
# stemmed train dataset 
stemmed_train_binary_df=dataset_after_stemming(binary_train_df)
stemmed_train_binary_df.head()

,Unique ID,Post,Labels Set
0,1,मेर देश क हिन्द बहुत निराल है कुछ त पक्क राम भक्त है और कुछ बाबर क साल है जय श्र राम,hostile
1,2,सरकार हमेश स किसान क कम क बढ़ क ल न न स्कीम ल रह है ताक उन पर ज्याद आर्थिक बोझ पड़,non-hostile
2,3,सुशांत न ज बिजनेस डील जून क क थ व डील दीपेश क सुशांत क हत्य क दिन ह क्य याद आ देख पूछ है भारत अर्नब क साथ रिपब्लिक भारत पर LIVE,non-hostile
3,4,prabhav218 साल जेएनय छाप कमिन लोग हिन्द क यह कह है क संविधान सबक बराबर अधिकार देत है सच्च यह है क यह बराबर अधिकार नह देत है,hostile
4,5,unlock4guidelines अनलॉक क ल गाइडलाइन्स जार सितंबर स देशभर म मेट्र सेव शुर ह 21 सितंबर क बाद रैल और बाक फंक्शन म 100 लोग क इजाजत कंटेनमेंट जोन म को छूट नह सिनेमाहॉल अभ बंद रह स 12व क छात्र 21 सितंबर क बाद स्कूल ज सक,non-hostile


In [ ]:
# stemmed validate dataset
stemmed_validate_binary_df=dataset_after_stemming(binary_validate_df)
stemmed_validate_binary_df.head()

,Unique ID,Post,Labels Set
0,1,दृढ़ इच्छ शक्त स परिपूर्ण प्रणबद क ल देशहित सर्वोच्च रह उनक निधन हम सब क ल अपूरणीय क्षत है ईश्वर दिवंगत आत्म क अप श्रीचरण म स्थान द शोक संतप्त परिजन क प्रत संवेद ऊं शांत,non-hostile
1,2,भारतीय जन पार्ट rss वाल इत गिर हु हैं जह मैं रह हूं वह मेर जासूस कर रह है उसक जासूस क पहचान मुझ अच्छ तरह है rss बीजेप वाल क जासूस दिल्ल म कौन है उत्तर प्रदेश म कौन है हरियाण राजस्थान म कौन है सबक पहचान है मुझ मेर नजर स बच नह सक ह,hostile
2,3,कोरोन स निपट क तैयार दिल्ल म 10 हजार बेड वाल दुनिय क सबस बड़ कोविड केयर सेंटर शुर राजनाथ शाह न डीआरडी क हजार बेड वाल सेंटर क भ उद्घाटन किय Delhi ArvindKejriwal rajnathsingh AmitShah DRDO India WHO crpfindia ITBP official,non-hostile
3,4,गवर्नर कॉन्फ्रेंस म PM मोद बोल शिक्ष नीत म सरकार क दखल कम होन चाह,non-hostile
4,5,यूप गाजीपुर म Toilet घोटाल प्रधान सचिव न किय लाख क गबन मुर्द क नाम पर बनव डाल शौचालय UP,non-hostile


In [ ]:
# ner + stemmed train 
ner_stemmed_train_binary_df=dataset_after_stemming(ner_binary_train)
ner_stemmed_train_binary_df.head()

,Unique ID,Post,Labels Set
0,1,देश क हिन्द बहुत निराल है कुछ त पक्क भक्त है और कुछ क साल है जय राम,hostile
1,2,हमेश स किसान क कम क बढ़ क ल न न स्कीम ल रह है ताक उन पर ज्याद आर्थिक बोझ पड़,non-hostile
2,3,ज बिजनेस डील जून क क थ व डील दीपेश क क हत्य क दिन ह क्य याद आ देख पूछ है अर्नब क साथ रिपब्लिक पर LIVE,non-hostile
3,4,छाप कमिन लोग क यह कह है क संविधान सबक बराबर अधिकार देत है सच्च यह है क यह बराबर अधिकार नह देत है,hostile
4,5,अनलॉक क ल गाइडलाइन्स जार सितंबर स म मेट्र सेव शुर ह 21 सितंबर क बाद रैल और बाक फंक्शन म 100 लोग क इजाजत कंटेनमेंट जोन म को छूट नह सिनेमाहॉल अभ बंद रह स 12व क छात्र 21 सितंबर क बाद स्कूल ज सक,non-hostile


In [ ]:
# ner + stemmed validate
ner_stemmed_validate_binary_df=dataset_after_stemming(ner_binary_validate)
ner_stemmed_validate_binary_df.head()

,Unique ID,Post,Labels Set
0,1,इच्छ शक्त स परिपूर्ण प्रणबद क ल देशहित सर्वोच्च रह उनक निधन हम सब क ल अपूरणीय क्षत है ईश्वर दिवंगत आत्म क अप श्रीचरण म स्थान द शोक संतप्त परिजन क प्रत संवेद ऊं शांत,non-hostile
1,2,जन rss वाल इत गिर हु हैं जह मैं रह हूं वह मेर जासूस कर रह है उसक जासूस क पहचान मुझ अच्छ तरह है rss वाल क जासूस म कौन है प्रदेश म कौन है राजस्थान म कौन है सबक पहचान है मुझ मेर नजर स बच नह सक ह,hostile
2,3,स निपट क तैयार म 10 हजार बेड वाल दुनिय क सबस बड़ कोविड केयर सेंटर शुर शाह न क हजार बेड वाल सेंटर क भ उद्घाटन किय ArvindKejriwal rajnathsingh AmitShah crpfindia ITBP official,non-hostile
3,4,कॉन्फ्रेंस म PM बोल शिक्ष नीत म सरकार क दखल कम होन चाह,non-hostile
4,5,म Toilet घोटाल प्रधान सचिव न किय लाख क गबन मुर्द क नाम पर बनव डाल शौचालय UP,non-hostile


# **DOWNLOADING ALL THE PRE-PROCESSED DATA:**

In [ ]:
"""
#cleaned_train
cleaned_train_df.to_excel("cleaned_train_data.xlsx") 
from google.colab import files
files.download('cleaned_train_data.xlsx')

#cleaned validate
cleaned_validate_df.to_excel("cleaned_validate_data.xlsx") 
from google.colab import files
files.download('cleaned_validate_data.xlsx')

# emojis removed dataset 

train_emoji.to_excel("train_emoji.xlsx") 
from google.colab import files
files.download('train_emoji.xlsx')

validate_emoji.to_excel("validate_emoji.xlsx") 
from google.colab import files
files.download('validate_emoji.xlsx')

# punctuations removed dataset 

train_punctuations.to_excel("train_punctuations.xlsx") 
from google.colab import files
files.download('train_punctuations.xlsx')

validate_punctuations.to_excel("validate_punctuations.xlsx") 
from google.colab import files
files.download('validate_punctuations.xlsx')

#hyperlinks removed dataset 

train_hyperlinks.to_excel("train_hyperlinks.xlsx") 
from google.colab import files
files.download('train_hyperlinks.xlsx')

validate_hyperlinks.to_excel("validate_hyperlinks.xlsx") 
from google.colab import files
files.download('validate_hyperlinks.xlsx')

# emojis removed dataset 

train_newlines.to_excel("train_newlines.xlsx") 
from google.colab import files
files.download('train_newlines.xlsx')

validate_newlines.to_excel("validate_newlines.xlsx") 
from google.colab import files
files.download('validate_newlines.xlsx')

#binary_train
binary_train_df.to_excel("binary_train_data.xlsx") 
from google.colab import files
files.download('binary_train_data.xlsx')

#binary validate
binary_validate_df.to_excel("binary_validate_data.xlsx") 
from google.colab import files
files.download('binary_validate_data.xlsx')

# 5 one hot vectors train
outputlabel5_train_df.to_excel("output5hot_train_data.xlsx") 
from google.colab import files
files.download('output5hot_train_data.xlsx')

# 5 one hot vectors validate 
outputlabel5_validate_df.to_excel("output5hot_validate_data.xlsx") 
from google.colab import files
files.download('output5hot_validate_data.xlsx')

# 4 one hot vectors train
outputlabel4_train_df.to_excel("output4hot_train_data.xlsx") 
from google.colab import files
files.download('output4hot_train_data.xlsx')

# 4 one hot vectors validate
outputlabel4_validate_df.to_excel("output4hot_validate_data.xlsx") 
from google.colab import files
files.download('output4hot_validate_data.xlsx')

# fake train 
fake_train.to_excel("fake_train.xlsx") 
from google.colab import files
files.download('fake_train.xlsx')

# fake validate
fake_validate.to_excel("fake_validate.xlsx") 
from google.colab import files
files.download('fake_validate.xlsx')

# hate train
hate_train.to_excel("hate_train.xlsx") 
from google.colab import files
files.download('hate_train.xlsx')

# hate validate 
hate_validate.to_excel("hate_validate.xlsx") 
from google.colab import files
files.download('hate_validate.xlsx')

# defamation train
defamation_train.to_excel("defamation_train.xlsx") 
from google.colab import files
files.download('defamation_train.xlsx')

# defamation validate 
defamation_validate.to_excel("defamation_validate.xlsx") 
from google.colab import files
files.download('defamation_validate.xlsx')

# offensive train 
offensive_train.to_excel("offensive_train.xlsx") 
from google.colab import files
files.download('offensive_train.xlsx')

# offensive validate 
offensive_validate.to_excel("offensive_validate.xlsx") 
from google.colab import files
files.download('offensive_validate.xlsx')

# named entity recognition train data
ner_binary_train.to_excel("NER_train.xlsx") 
from google.colab import files
files.download('NER_train.xlsx')

# named entity recognition validate data
ner_binary_validate.to_excel("NER_validate.xlsx") 
from google.colab import files
files.download('NER_validate.xlsx')

# stemmed train data
stemmed_train_binary_df.to_excel("stemmed_train.xlsx") 
from google.colab import files
files.download('stemmed_train.xlsx')

# stemmed validate data
stemmed_validate_binary_df.to_excel("stemmed_validate.xlsx") 
from google.colab import files
files.download('stemmed_validate.xlsx')

# NER stemmed train data
ner_stemmed_train_binary_df.to_excel("ner_stemmed_train.xlsx") 
from google.colab import files
files.download('stemmed_train.xlsx')

# NER stemmed validate data
ner_stemmed_validate_binary_df.to_excel("ner_stemmed_validate.xlsx") 
from google.colab import files
files.download('stemmed_validate.xlsx')
"""

'\n#cleaned_train\ncleaned_train_df.to_excel("cleaned_train_data.xlsx") \nfrom google.colab import files\nfiles.download(\'cleaned_train_data.xlsx\')\n\n#cleaned validate\ncleaned_validate_df.to_excel("cleaned_validate_data.xlsx") \nfrom google.colab import files\nfiles.download(\'cleaned_validate_data.xlsx\')\n\n# emojis removed dataset \n\ntrain_emoji.to_excel("train_emoji.xlsx") \nfrom google.colab import files\nfiles.download(\'train_emoji.xlsx\')\n\nvalidate_emoji.to_excel("validate_emoji.xlsx") \nfrom google.colab import files\nfiles.download(\'validate_emoji.xlsx\')\n\n# punctuations removed dataset \n\ntrain_punctuations.to_excel("train_punctuations.xlsx") \nfrom google.colab import files\nfiles.download(\'train_punctuations.xlsx\')\n\nvalidate_punctuations.to_excel("validate_punctuations.xlsx") \nfrom google.colab import files\nfiles.download(\'validate_punctuations.xlsx\')\n\n#hyperlinks removed dataset \n\ntrain_hyperlinks.to_excel("train_hyperlinks.xlsx") \nfrom google.cola